In [1]:
!pip install -q tensorflow
!pip install -q keras


## Imports libs
import os
from tensorflow import keras

In [3]:
 !pip install -q tensorflow-model-optimization

In [4]:
import tempfile
import os

import tensorflow as tf

from tensorflow import keras

In [5]:
import tensorflow as tf
import numpy as np
import tensorflow_model_optimization as tfmot

import tempfile

In [6]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [2]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

NameError: name 'tf' is not defined

In [1]:
predictions = model(x_train[:10]).numpy()
predictions

NameError: name 'model' is not defined

In [10]:
tf.nn.softmax(predictions).numpy()

array([[0.06769237, 0.08063873, 0.06308316, 0.0444681 , 0.14171338,
        0.10558186, 0.15701105, 0.06409926, 0.04961228, 0.22609982]],
      dtype=float32)

In [11]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [12]:
loss_fn(y_train[:1], predictions).numpy()

2.2482686

In [13]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [14]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 2s 987us/step - loss: 0.3017 - accuracy: 0.9125
Epoch 2/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1492 - accuracy: 0.9557
Epoch 3/5
1875/1875 [==============================] - 2s 922us/step - loss: 0.1133 - accuracy: 0.9647
Epoch 4/5
1875/1875 [==============================] - 2s 906us/step - loss: 0.0919 - accuracy: 0.9715
Epoch 5/5
1875/1875 [==============================] - 2s 895us/step - loss: 0.0795 - accuracy: 0.9752


In [15]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0768 - accuracy: 0.9752 - 334ms/epoch - 1ms/step


[0.07683579623699188, 0.9751999974250793]

In [17]:
base_model=model

quant_aware_model = tfmot.quantization.keras.quantize_model(base_model)
quant_aware_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer (QuantizeLay  (None, 28, 28)           3         
 er)                                                             
                                                                 
 quant_flatten (QuantizeWrap  (None, 784)              1         
 perV2)                                                          
                                                                 
 quant_dense (QuantizeWrappe  (None, 128)              100485    
 rV2)                                                            
                                                                 
 quant_dropout (QuantizeWrap  (None, 128)              1         
 perV2)                                                          
                                                                 
 quant_dense_1 (QuantizeWrap  (None, 10)               1

In [35]:
LastValueQuantizer = tfmot.quantization.keras.quantizers.LastValueQuantizer
MovingAverageQuantizer = tfmot.quantization.keras.quantizers.MovingAverageQuantizer

class DefaultDenseQuantizeConfig(tfmot.quantization.keras.QuantizeConfig):
    # Configure how to quantize weights.
    def get_weights_and_quantizers(self, layer):
      return [(layer.kernel, LastValueQuantizer(num_bits=8, symmetric=True, narrow_range=False, per_axis=False))]

    # Configure how to quantize activations.
    def get_activations_and_quantizers(self, layer):
      return [(layer.activation, MovingAverageQuantizer(num_bits=8, symmetric=False, narrow_range=False, per_axis=False))]

    def set_quantize_weights(self, layer, quantize_weights):
      # Add this line for each item returned in `get_weights_and_quantizers`
      # , in the same order
      layer.kernel = quantize_weights[0]

    def set_quantize_activations(self, layer, quantize_activations):
      # Add this line for each item returned in `get_activations_and_quantizers`
      # , in the same order.
      layer.activation = quantize_activations[0]

    # Configure how to quantize outputs (may be equivalent to activations).
    def get_output_quantizers(self, layer):
      return []

    def get_config(self):
      return {}

In [36]:
quantize_annotate_layer = tfmot.quantization.keras.quantize_annotate_layer
quantize_annotate_model = tfmot.quantization.keras.quantize_annotate_model
quantize_scope = tfmot.quantization.keras.quantize_scope

class FixedRangeQuantizer(tfmot.quantization.keras.quantizers.Quantizer):
  """Quantizer which forces outputs to be between -1 and 1."""

  def build(self, tensor_shape, name, layer):
    # Not needed. No new TensorFlow variables needed.
    return {}

  def __call__(self, inputs, training, weights, **kwargs):
    return tf.keras.backend.clip(inputs, -1.0, 1.0)

  def get_config(self):
    # Not needed. No __init__ parameters to serialize.
    return {}


class ModifiedDenseQuantizeConfig(DefaultDenseQuantizeConfig):
    # Configure weights to quantize with 4-bit instead of 8-bits.
    def get_weights_and_quantizers(self, layer):
      # Use custom algorithm defined in `FixedRangeQuantizer` instead of default Quantizer.
      return [(layer.kernel, FixedRangeQuantizer())]

In [62]:
model = quantize_annotate_model(tf.keras.Sequential([
   
    keras.layers.InputLayer(input_shape=(28, 28)),
    
    keras.layers.Reshape(target_shape=(28, 28, 1)),
    keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    # Pass in modified `QuantizeConfig` to modify this `Dense` layer.
   quantize_annotate_layer(tf.keras.layers.Dense(129,), ModifiedDenseQuantizeConfig()),
    
]))

# `quantize_apply` requires mentioning `ModifiedDenseQuantizeConfig` with `quantize_scope`:
with quantize_scope(
  {'ModifiedDenseQuantizeConfig': ModifiedDenseQuantizeConfig}):
  # Use `quantize_apply` to actually make the model quantization aware.
  quant_aware_model = tfmot.quantization.keras.quantize_apply(model)

quant_aware_model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer_8 (QuantizeL  (None, 28, 28)           3         
 ayer)                                                           
                                                                 
 quant_reshape_5 (QuantizeWr  (None, 28, 28, 1)        1         
 apperV2)                                                        
                                                                 
 quant_conv2d_7 (QuantizeWra  (None, 26, 26, 12)       147       
 pperV2)                                                         
                                                                 
 quant_max_pooling2d_7 (Quan  (None, 13, 13, 12)       1         
 tizeWrapperV2)                                                  
                                                                 
 quant_flatten_16 (QuantizeW  (None, 2028)           

In [63]:
# `quantize_model` requires a recompile.
quant_aware_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

quant_aware_model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer_8 (QuantizeL  (None, 28, 28)           3         
 ayer)                                                           
                                                                 
 quant_reshape_5 (QuantizeWr  (None, 28, 28, 1)        1         
 apperV2)                                                        
                                                                 
 quant_conv2d_7 (QuantizeWra  (None, 26, 26, 12)       147       
 pperV2)                                                         
                                                                 
 quant_max_pooling2d_7 (Quan  (None, 13, 13, 12)       1         
 tizeWrapperV2)                                                  
                                                                 
 quant_flatten_16 (QuantizeW  (None, 2028)           

In [64]:
quant_aware_model.fit(x_train, y_train,  batch_size=, epochs=5)

Epoch 1/5
469/469 [==============================] - 6s 11ms/step - loss: 0.5484 - accuracy: 0.8635
Epoch 2/5
469/469 [==============================] - 5s 12ms/step - loss: 0.1874 - accuracy: 0.9474
Epoch 3/5
469/469 [==============================] - 5s 12ms/step - loss: 0.1254 - accuracy: 0.9653
Epoch 4/5
469/469 [==============================] - 6s 12ms/step - loss: 0.0978 - accuracy: 0.9727
Epoch 5/5
469/469 [==============================] - 6s 13ms/step - loss: 0.0828 - accuracy: 0.9767


In [74]:
quant_aware_model.evaluate(x_test,  y_test, verbose=0)

[0.07915325462818146, 0.9747999906539917]

In [70]:
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [76]:
_, baseline_model_accuracy = base_model.evaluate(
    test_images, test_labels, verbose=0)

_, quant_aware_model_accuracy = quant_aware_model.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Quantize test accuracy:', quant_aware_model_accuracy)

Baseline test accuracy: 0.9757999777793884
Quantize test accuracy: 0.9585999846458435


In [77]:
converter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model)


converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_tflite_model9 = converter.convert()

INFO:tensorflow:Assets written to: C:\Windows\TEMP\tmpg36y0kn6\assets


INFO:tensorflow:Assets written to: C:\Windows\TEMP\tmpg36y0kn6\assets
C:\Users\Meskat Jahan\anaconda3\lib\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


In [78]:
import numpy as np

def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy

In [79]:
interpreter = tf.lite.Interpreter(model_content=quantized_tflite_model9)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

print('Quant TFLite test_accuracy:', test_accuracy)
print('Quant TF test accuracy:', quant_aware_model_accuracy)

Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.
Evaluated on 4000 results so far.
Evaluated on 5000 results so far.
Evaluated on 6000 results so far.
Evaluated on 7000 results so far.
Evaluated on 8000 results so far.
Evaluated on 9000 results so far.


Quant TFLite test_accuracy: 0.9579
Quant TF test accuracy: 0.9585999846458435


In [80]:
interpreter = tf.lite.Interpreter(model_content=quantized_tflite_model9)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

print('Quant TFLite test_accuracy:', test_accuracy)
print('Quant TF test accuracy:', quant_aware_model_accuracy)

Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.
Evaluated on 4000 results so far.
Evaluated on 5000 results so far.
Evaluated on 6000 results so far.
Evaluated on 7000 results so far.
Evaluated on 8000 results so far.
Evaluated on 9000 results so far.


Quant TFLite test_accuracy: 0.9579
Quant TF test accuracy: 0.9585999846458435


In [82]:
# Create float TFLite model.
float_converter = tf.lite.TFLiteConverter.from_keras_model(base_model)
float_tflite_model10 = float_converter.convert()


# Measure sizes of models.
_, float_file = tempfile.mkstemp('.tflite')
_, quant_file = tempfile.mkstemp('.tflite')

with open(quant_file, 'wb') as f:
  f.write(quantized_tflite_model9)

with open(float_file, 'wb') as f:
  f.write(float_tflite_model10)

print("Float model in Mb:", os.path.getsize(float_file) / float(2**20))
print("Quantized model in Mb:", os.path.getsize(quant_file) / float(2**20))

INFO:tensorflow:Assets written to: C:\Windows\TEMP\tmp6v1u8vyy\assets


INFO:tensorflow:Assets written to: C:\Windows\TEMP\tmp6v1u8vyy\assets


Float model in Mb: 0.389923095703125
Quantized model in Mb: 0.25444793701171875
